# Pregunta 3  Tarea 2
# Segunda Parte: Correr Inversión de Minimos Cuadrados Regularizados

## GF5013 - Metodos Inversos Aplicados a la Geofisica
## Primavera 2019
### Prof. Francisco Hernán Ortega Culaciati 
### ortega.francisco@uchile.cl
### Departamento de Geofísica - FCFM
### Universidad de Chile
### Octubre de 2019

Nombre Alumno: XXXX YYYY

# Desarrollo ecuaciones P3 (b) : Resolución del problema general de mínimos cuadrados regularizado

escriba aquí la explicación (con ecuaciones) 

In [ ]:
import sys, os # modifique el pathTarea2
pathTarea2 = '../'
pathPaquetesPython = pathTarea2 + '/PaquetesPython'
TomoDataFilename = pathTarea2 + '/datosTomo/datosTomografia.txt'

sys.path.append(os.path.abspath(pathPaquetesPython))




In [ ]:
import matplotlib
# descomentar esto es para hacer los plots interactivos en el notebook
%matplotlib notebook 
# descomentar esto es para hacer los plots afuera del notebook
#%matplotlib tk  
import matplotlib.pyplot as plt
import numpy as NP

from Tarea2.TomoData import readTravelTimeData as readTTdata
from Tarea2 import TomographyModel as Tmodel
import Tarea2
import GF5013
import pickle

## Cargar los datos necesarios para la inversión

In [ ]:
PickleFilename = "Tomo_pre_inversion_data.pickle"
PickleFile = open(PickleFilename, 'rb')
dataPreInversion = pickle.load(PickleFile)
PickleFile.close()
G = dataPreInversion['G']
d_syn = dataPreInversion['d_syn']
Cx = dataPreInversion['Cx']
m_syn = dataPreInversion['m_syn']
TTdata = dataPreInversion['TTdata']
mesh = dataPreInversion['mesh'] 
H = dataPreInversion['H'] 
Wh = dataPreInversion['Wh']
ho = dataPreInversion['ho']
inv_Cx = dataPreInversion['inv_Cx']
Wx = dataPreInversion['Wx'] 

Ndata, Npar = # COMPLETAR

# semilla del generador de numeros aleatorios (use su fecha de nacimiento DDMMYYYY)
seed = 28101979
NP.random.seed(seed)

## Definir parámetros para la inversión

In [ ]:
TypeOfData = 'Syn' # set to 'Syn' if doing inversion with synthetic data
                   # set to 'Tomo' if doing the inversion with Tomography experiment data (from the datafile)

if TypeOfData == 'Syn': # does the inversion with sythetic data with added noise
    # se usará los datos sintéticos más un ruido gaussiano con desviación estándar A = 1 [ms]
    A = 1
    NP.random.seed(seed)
    ruido = A * NP.random.randn(Ndata,1)
    d = d_syn + ruido

elif TypeOfData == 'Tomo': # does the inversion using the data from the Tomography experiment ()
    # si por el contrario desea hacer la inversión con datos reales, descomentar la linea siguiente.
    d = TTdata['TV'].reshape((Ndata,1))

else:
    raise ValueError('Type of data does not exist...')
    
    
# definir el rango de valores para la constante de amortiguamiento
epsilonMin = 1E-4
epsilonMax = 1E4
Nepsilons = 200 # cambie esto a un numero mas bajo (ej 30)  mientras esta programando los codigos, use 200 al final.
EPSILONS = NP.logspace(NP.log10(epsilonMin), NP.log10(epsilonMax), Nepsilons)



## Realizar la inversión para todos los epsilons.


In [ ]:
# para no ocupar demasiada memoria se guardará sólo el modelo estimado (y no las matrices de covarianza a posteriori)
# para todos los valores de epsilon
m_estimados = NP.zeros((Npar, Nepsilons))

for i in range(0, Nepsilons):
    epsilon = EPSILONS[i]
    #print([i, epsilon])
    m_est = GF5013.optim.MinCuadRegWeights.MinCuadRegWeights(G, d, Wx, H, ho, Wh, epsilon)['m']
    m_estimados[:,i] = m_est.squeeze()

## Guardar los resultados de la inversión para procesarlos en otro notebook
(Solo necesito guardar las variables adicionales que se crearon en este notebook)


In [ ]:
# nombre del archivo "pickle" donde guardar los datos de interés
PickleFilename = "Tomo_Result_Inversion.pickle"

# variable a guardar es un diccionario con G, d y la malla
data2save = {}
data2save['H'] = H
data2save['ho'] = ho
data2save['Wh'] = Wh

data2save['Wx'] = Wx
data2save['d'] = d
data2save['G'] = G

data2save['EPSILONS'] = EPSILONS 
data2save['m_estimados'] = m_estimados 

data2save['mesh'] = mesh

# save the dictionray in the file.
PickleFile = open(PickleFilename, 'wb') # must be a binary file w: writable, b: binary
pickle.dump(data2save, PickleFile, pickle.HIGHEST_PROTOCOL)
PickleFile.close()



In [ ]:
# Aca se puede graficar una solución de la inversión, solo por deporte se los dejo!!!

In [ ]:
I_inversion = 148
F = 1./m_estimados[:, I_inversion]  
colorlim = [0, NP.max(F)]
fig = plt.figure(2)
fig.clear()
ax = plt.subplot(111)
fig, ax = Tarea2.Mesh.plotRectMeshField(F, mesh, fig, ax , colorlim = None, \
                                 title = "Modelo Estimado",\
                                 colorbar_label = 'Estimated Velocity ', xlabel = 'X [m]',\
                                 ylabel = 'Y [m]', colormap = 'gray')
fig.show()